<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/STT_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SpeechRecognition
!pip install pydub

In [ ]:
import pandas as pd
from tqdm import tqdm
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

# DATA

In [23]:
data_emotion = pd.read_csv('data_emotion.csv')
data_main = pd.read_csv('data_main.csv')

In [24]:
data_emotion

,Emotion,Path
0,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
1,sad,/content/drive/MyDrive/project/data_aihub/5/5e...
2,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
3,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
4,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
...,...,...
16551,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...
16552,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...
16553,surprise,/content/drive/MyDrive/project/data_aihub/4/5e...
16554,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...


In [25]:
data_main

,Path
0,/content/drive/MyDrive/project/aihub_data_main...
1,/content/drive/MyDrive/project/aihub_data_main...
2,/content/drive/MyDrive/project/aihub_data_main...
3,/content/drive/MyDrive/project/aihub_data_main...
4,/content/drive/MyDrive/project/aihub_data_main...
...,...
409,/content/drive/MyDrive/project/aihub_data_main...
410,/content/drive/MyDrive/project/aihub_data_main...
411,/content/drive/MyDrive/project/aihub_data_main...
412,/content/drive/MyDrive/project/aihub_data_main...


# STT

## emotion

In [26]:
recognizer = sr.Recognizer()

In [27]:
def speech_to_text(file_path):
    try:
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [28]:
data_emotion['Text'] = [speech_to_text(path) for path in tqdm(data_emotion['Path'])]

100%|██████████| 16556/16556 [3:42:29<00:00,  1.24it/s]


In [29]:
data_emotion

,Emotion,Path,Text
0,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,나 결국 여자 친구와 헤어졌어
1,sad,/content/drive/MyDrive/project/data_aihub/5/5e...,이 회사 들어올 때 얼마나 힘들었는지 알잖아 조금만 더 버텨 보고 싶어
2,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,회초리도 맞고 빠따도 맞아서 혼났지
3,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,거의 다 떨어져가
4,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,평소에는 너무 잘 먹고 사이 좋았는데 마지막에 지쳤나 봐
...,...,...,...
16551,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...,그런 영상도 있었어
16552,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...,야 나 이벤트 당첨됐어
16553,surprise,/content/drive/MyDrive/project/data_aihub/4/5e...,엄청 놀랐어
16554,surprise,/content/drive/MyDrive/project/data_aihub/5_2/...,갑자기 해피가 목줄을 끊고 막 뛰어 가잖아 그래서 못 잡을까 봐 너무 놀랐어


In [30]:
data_emotion.to_csv('emotion_text.csv', index = False, encoding = 'utf-8-sig')

## main

### STT - 50개씩 나눠서

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
def split_audio_on_silence(file_path):
  sound = AudioSegment.from_wav(file_path)

  chunks = split_on_silence(sound, min_silence_len = 1000, silence_thresh = -40)
  return chunks

In [ ]:
def speech_to_text(audio_chunk):
    try:
        audio_data = audio_chunk.export(format = "wav")
        with sr.AudioFile(audio_data) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [ ]:
def process_audio(file_path):
    chunks = split_audio_on_silence(file_path)
    text_results = []

    for i, chunk in enumerate(chunks):
        text = speech_to_text(chunk)
        text_results.append(f"Chunk {i+1}: {text}")

    return text_results

In [ ]:
data_list = []
for file_path in tqdm(data_main['Path'][0:50], desc = "Processing 1-50 files"):
    stt_results = process_audio(file_path)
    for result in stt_results:
        data_list.append({'Path': file_path, 'STT 결과': result})

Processing 1-50 files: 100%|██████████| 50/50 [1:45:08<00:00, 126.18s/it]


In [ ]:
data_main = pd.DataFrame(data_list)
data_main

,Path,STT 결과
0,/content/drive/MyDrive/project/aihub_data_main...,Chunk 1: 미영
1,/content/drive/MyDrive/project/aihub_data_main...,Chunk 2: 아이
2,/content/drive/MyDrive/project/aihub_data_main...,Chunk 3: 단짝 미애와 아이는 노래하며 웁니다
3,/content/drive/MyDrive/project/aihub_data_main...,Chunk 4: 나를 위해 나란히 누워
4,/content/drive/MyDrive/project/aihub_data_main...,Chunk 5: 랑랑이 노래합니다
...,...,...
3583,/content/drive/MyDrive/project/aihub_data_main...,Chunk 317: 천안 지금 팔이랑 시차
3584,/content/drive/MyDrive/project/aihub_data_main...,Chunk 318: 한국 증시 알려
3585,/content/drive/MyDrive/project/aihub_data_main...,Chunk 319: 한글날 몇
3586,/content/drive/MyDrive/project/aihub_data_main...,Chunk 320: 현재 날씨 알려


In [ ]:
data_main.to_csv('main_text_1.csv', index = False, encoding = 'utf-8-sig')